### EMTOOLS -- Validation
**Description:** This notebook is for qualitative and quantitative validation of automatically segmented images in the pipeline. It takes a raw input

**Author:** Philip Ruthig, Paul Flechsig Institute, Center of Neuropathology and Brain Research Leipzig

**Contact:** philip.ruthig@medizin.uni-leipzig.de // philip.ruthig@gmail.com

**Publication:**
Please contact me if you want to use this code for any publication.

#### Correlations and errors quantified between predicted and validation (ground gruth) data:
- Area
- Eccentricity
- major axis length
- minor axis length
- g Ratio
- Percentage of False Positives / Negatives
- Bland-Altman plots of prediction and validation data
- IoU (Intersection over Union) Score (=Jaccard) 
- Dice Coefficient (=F1 Score)

In [ ]:
# import dependencies
import numpy as np
import tifffile as tf
import scipy.ndimage as ndi
import matplotlib.pyplot as plt
import os
import pandas as pd
import math as m
import skimage
import cv2
import scipy
from skimage.measure import regionprops, regionprops_table
from skimage.morphology import disk
import colorcet as cc
import platform
%matplotlib inline

In [ ]:
def resolve_undersegmentation(outer_labels,inner_labels):
    '''
    resolves undersegmented cells with 'kissing' cells. This function assumes that inner_labels never touch, but outer_labels do.
    Each corresponding outer area must touch each corresponding inner area.

    outer_labels = outer labels with undersegmented kissing cells that need to be seperated
    inner_labels = inner labels without undersegmented kissing cells

    returns: Two arrays of the same shape, with uniform labels across both images. 
    '''
    from skimage.segmentation import watershed
    from skimage.feature import peak_local_max
    outer_labels = outer_labels.astype('bool')
    inner_labels = inner_labels.astype('bool')
    # do distance transformation of combined binary image of outer+inner
    seg_dist = ndi.distance_transform_edt(outer_labels+inner_labels)
    # Generate the markers as local maxima of the distance to the background
    coords = peak_local_max(seg_dist, footprint=np.ones((3, 3)), labels=ndi.label(inner_labels)[0], num_peaks_per_label=1)
    # initializie empty mask 
    mask = np.zeros(seg_dist.shape, dtype=bool)
    # insert maximum points into the empty array
    mask[tuple(coords.T)] = True
    # label each maximum
    markers, n = ndi.label(mask)
    # print("number of cells: " + str(n))
    # perform watershed on outer_labels and inner_labels combined
    outer_cells = watershed(-seg_dist, markers, mask=(outer_labels+inner_labels))
    inner_cells = np.copy(outer_cells)
    # sort the cells (which are now labeled with the same label inner and outer) back into inner and outer labels
    inner_cells[inner_labels==0] = 0
    outer_cells[inner_labels==True] = 0
    return outer_cells,inner_cells

def keep_largest_structure(boolean_array):
    from scipy import ndimage
    # Label each connected component in the boolean array
    labeled_array, num_features = ndimage.label(boolean_array)

    # Calculate the size of each labeled component
    component_sizes = np.bincount(labeled_array.ravel())

    # Find the index of the largest component
    largest_component_index = np.argmax(component_sizes[1:]) + 1

    # Create a boolean mask to keep only the largest component
    largest_component_mask = labeled_array == largest_component_index

    # Apply the mask to the boolean array
    boolean_array[largest_component_mask] = True
    boolean_array[~largest_component_mask] = False

    return boolean_array

def calculate_iou(predicted_mask, ground_truth_mask):
    # IoU Score is equivalent to Jaccard
    # calculate intersection and union masks
    intersection = predicted_mask & ground_truth_mask # element-wise logical AND
    union = predicted_mask | ground_truth_mask #element-wise logical OR

    # calculate the number of pixels in the intersection and union masks
    n_intersection = intersection.sum()
    n_union = union.sum()

    # calculate the IoU score
    iou = n_intersection / n_union

    return iou

def calculate_dice(predicted_mask, ground_truth_mask):
    # This Dice coefficient is equivalent to F1 score.
    dice = np.sum(predicted_mask[ground_truth_mask==True]==True)*2.0 / (np.sum(predicted_mask[predicted_mask==True]==True) + np.sum(ground_truth_mask[ground_truth_mask==True]==True))
    return dice

def convert_windows_to_linux_path(windows_path):
    return windows_path.replace('\\', '/')

def mk_dir(directory_path):
    if not os.path.exists(directory_path):
        os.makedirs(directory_path)

# define qualitative colormap
glasbey = cc.cm.glasbey_dark_r
glasbey.set_under(color="black")

In [ ]:
prediction_path = os.getcwd() + r"/4_val/prediction.png" #prediction
validation_path = os.getcwd() + r"/4_val/validation.png" #ground truth
save_path = os.getcwd() + r"/5_val_results/"

crop = False

In [ ]:
mk_dir(save_path)
# open data
img = cv2.imread(prediction_path)
img_val = cv2.imread(validation_path)
val_inner = img_val[:,:,2]
val_outer = img_val[:,:,0]

# lose two dimensions, since its a grayscale img 
img = img[:,:,0]

plt.hist(img.ravel(), bins=256, range=(0, 255),)
plt.show()

# split different labels
pred_outer = np.zeros_like(img)
pred_inner = np.zeros_like(img)
pred_outer[img>50] = 1
pred_inner[img<40] = 1
pred_inner[img<24] = 0

fig, axs = plt.subplots(ncols=2,figsize=(12,12))
axs[0].imshow(pred_outer[0:1000,0:1000], cmap='gray')
axs[1].imshow(pred_inner[0:1000,0:1000], cmap='gray')
axs[0].set_title('Outer - raw output')
axs[1].set_title('Inner - raw output')
plt.show()

# binary opening to get rid of small speckles
# pred_inner = ndi.binary_opening(pred_inner,structure=disk(2))
pred_inner = ndi.binary_opening(pred_inner,structure=disk(5))

# fill holes
pred_inner = ndi.binary_fill_holes(pred_inner)

# binary opening to get rid of small speckles
# pred_inner = ndi.binary_opening(pred_inner,structure=disk(6))
pred_inner = ndi.binary_opening(pred_inner,structure=disk(7))

plt.imshow(pred_inner[0:1000,0:1000],interpolation='None',cmap='gray')
plt.show()

In [ ]:
#crop
if crop == True:
    crop_value = 2000
    pred_inner = pred_inner[0:crop_value,0:crop_value]
    pred_outer = pred_outer[0:crop_value,0:crop_value]
    val_inner = val_inner[0:crop_value,0:crop_value]
    val_outer = val_outer[0:crop_value,0:crop_value]

# fill inner to outer label to make sure they touch
pred_inner = ndi.binary_dilation(pred_inner,structure=disk(5))
pred_inner[pred_outer==True]=0

# binary opening to get rid of small speckles
pred_inner = ndi.binary_opening(pred_inner,structure=disk(2))

# fill holes
pred_inner = ndi.binary_fill_holes(pred_inner)

# binary opening to get rid of small speckles
pred_inner = ndi.binary_opening(pred_inner,structure=disk(6))

# dilate inner, then restrict it to everywhere where outer isnt true. 
# This is to make sure they are in contact and can be seperated by watershed later on
pred_inner = ndi.binary_dilation(pred_inner,structure=disk(3))
pred_inner[pred_outer==True]=False

#re-label so the labels are uniform between both channels.
pred_outer_labeled,pred_inner_labeled = resolve_undersegmentation(pred_outer.astype('bool'),pred_inner.astype('bool'))
plt.title('prediction outer')
plt.imshow(pred_outer_labeled,cmap=glasbey,interpolation='None',vmin=0.1,vmax=pred_outer_labeled.max())
plt.show()
plt.title('prediction inner')
plt.imshow(pred_inner_labeled,cmap=glasbey,interpolation='None',vmin=0.1,vmax=pred_outer_labeled.max())
plt.show()

n_pred_cells_raw = np.max(pred_inner_labeled)
print(n_pred_cells_raw)

In [ ]:
print(len(np.unique(pred_outer_labeled)))
print(len(np.unique(pred_inner_labeled)))
print(np.unique(pred_outer_labeled))
print(np.unique(pred_inner_labeled))

In [ ]:
n_labeled_without_outer = 0
n_labeled_without_inner = 0

# iterate through np.unique(pred_inner_labeled), delete all cells that do not have axon or myelin.
for id in np.unique(pred_inner_labeled):
    # check if id is in pred_outer_labeled
    if id not in pred_outer_labeled:
        # if not, delete in pred_inner_labeled
        pred_inner_labeled[pred_inner_labeled==id] = 0
        print("deleted inner label #" + str(id))
        n_labeled_without_outer += 1

for id in np.unique(pred_outer_labeled):
    if id not in pred_inner_labeled:
        # if not, delete in pred_outer_labeled
        pred_outer_labeled[pred_outer_labeled==id] = 0
        print("deleted outer label #" + str(id))
        n_labeled_without_inner += 1

In [ ]:
pred_outer_labeled,pred_inner_labeled = resolve_undersegmentation(pred_outer_labeled.astype('bool'),pred_inner_labeled.astype('bool'))
plt.title('prediction outer')
plt.imshow(pred_outer_labeled,cmap=glasbey,interpolation='None',vmin=0.1)
plt.savefig(save_path + "\pre_prediction_outer.png",dpi=500)
plt.show()
plt.title('prediction inner')
plt.imshow(pred_inner_labeled,cmap=glasbey,interpolation='None',vmin=0.1)
plt.savefig(save_path + "\pre_prediction_inner.png",dpi=500)
plt.show()
plt.title('prediction inner+outer')
plt.imshow(pred_inner_labeled+pred_outer_labeled,cmap=glasbey,interpolation='None',vmin=0.1)
plt.savefig(save_path + "\pre_prediction_inner_outer.png",dpi=500)
plt.show()

n_pred_cells_corrected = np.max(pred_inner_labeled)
print(n_pred_cells_corrected)

In [ ]:
print(len(np.unique(pred_outer_labeled)))
print(len(np.unique(pred_inner_labeled)))
print(np.unique(pred_outer_labeled))
print(np.unique(pred_inner_labeled))

In [ ]:
# fill inner to outer label to make sure they touch
# this is needed to ensure good watershed performance later
val_inner = ndi.binary_dilation(val_inner,structure=disk(5))
val_inner[val_outer==True]=0

#re-label so the labels are uniform between both channels.
val_outer_labeled,val_inner_labeled = resolve_undersegmentation(val_outer.astype('bool'),val_inner.astype('bool'))

plt.title('validation outer')
plt.imshow(val_outer_labeled,cmap=glasbey,interpolation='None',vmin=0.1)
plt.show()
plt.title('validation inner')
plt.imshow(val_inner_labeled,cmap=glasbey,interpolation='None',vmin=0.1)
plt.show()

In [ ]:
# at this point, some of the cells do not have myelin. 
# Since we only look for myelinated cells, these are filtered out here.
idx_list = []
for idx in np.unique(pred_inner_labeled):
    if (pred_outer_labeled==idx).max() == False:
        pred_inner_labeled[pred_inner_labeled==idx] = 0
        idx_list.append(idx)
print('deleted cells because they don\'t have myelin: ' + str(idx_list))

In [ ]:
pred_both_labeled = pred_inner_labeled+pred_outer_labeled
val_both_labeled = val_inner_labeled+val_outer_labeled

# plt.imshow(ndi.binary_dilation(pred_both_labeled[20:100,3250:3500,]))


In [ ]:
# np.count_nonzero(pred_both_labeled[20:60,3250:3290,])
# pred_both_labeled[20,3250]

#### Cleanup of predicted data
Remove segmented cells that touch the border of the image.

In [ ]:
n_cells_removed_border_pred = 0

# remove cells that intersect with the border
border_mask = np.zeros(pred_both_labeled.shape, dtype=bool)
border_mask = ndi.binary_dilation(border_mask,iterations=5, border_value=5)

for id in np.unique(pred_both_labeled):
    if id == 0:
        continue
    #keep only current cell as temporary mask
    current_id_mask = pred_both_labeled==id 

    # check if it overlaps with the border mask
    overlap = np.logical_and(border_mask,current_id_mask)
    overlap_n = np.sum(overlap)
    
    if overlap_n > 0:
        pred_both_labeled[pred_both_labeled==id] = 0
        print("deleted cell #" + str(id))
        n_cells_removed_border_pred += 1


In [ ]:
n_cells_removed_border_val = 0

# remove cells that intersect with the border
border_mask = np.zeros(val_inner_labeled.shape, dtype=bool)
border_mask = ndi.binary_dilation(border_mask,iterations=5, border_value=5)

for id in np.unique(val_inner_labeled):
    if id == 0:
        continue
    #keep only current cell as temporary mask
    current_id_mask = val_both_labeled==id 

    # check if it overlaps with the border mask
    overlap = np.logical_and(border_mask,current_id_mask)
    overlap_n = np.sum(overlap)

    if overlap_n > 0:
        val_both_labeled[val_both_labeled==id] = 0
        print("deleted cell #" + str(id))
        n_cells_removed_border_val += 1


In [ ]:
#### Post-Process filtering
# if there are multiple independent labels with the same labelling number,
# keep only the biggest one
for id in np.unique(pred_both_labeled):
    if id == 0:
        continue
    current_id_mask = pred_both_labeled==id 
    mask_labeled, n = ndi.label(current_id_mask)
    if n > 1:
        #delete all but the biggest one from this image and from pred_both_labeled
        current_id_kept = keep_largest_structure(current_id_mask)
        #correct the value of the boolean array to the current id
        current_id_kept = (current_id_kept*id).astype('uint16')
        #re-insert the remaining one to pred_both_labeled
        pred_both_labeled[pred_both_labeled==id] = 0
        pred_both_labeled = pred_both_labeled + current_id_kept        
        if plot_all == True:
            print ("Removed " + str(n-1) + " inner duplicates from id #" + str(id))

# if fibers are unreasonably small, remove them from further analysis.
for id in np.unique(pred_both_labeled):
    n_px = (np.count_nonzero(pred_both_labeled[pred_both_labeled==id]))
    if n_px < 5:
        pred_both_labeled[pred_both_labeled==id]=0
        print('deleted because smaller than 5 px: cell #' + str(id))

In [ ]:
# "mislabeled" data. not congruent IDs for now
plt.title('validation - inner + outer')
plt.imshow(val_both_labeled,cmap=glasbey,interpolation='None',vmin=0.1,vmax=np.max(pred_both_labeled))
plt.show()
plt.title('prediction - inner + outer')
plt.imshow(pred_both_labeled,cmap=glasbey,interpolation='None',vmin=0.1,vmax=np.max(pred_both_labeled))
plt.show()

In [ ]:
n_false_positive = 0
n_false_negative = 0

# Here, we start to define which cell (in predicted data) belongs to which other cell (in validation data). 
# Start by extracting centroids from the validation and prediction data.
# centroids_val = regionprops_table(label_image=val_inner_labeled,properties=('centroid',))
centroids_val = regionprops_table(label_image=val_both_labeled,properties=('centroid',))
centroids_val_x = centroids_val['centroid-0']
centroids_val_y = centroids_val['centroid-1']

# centroids_pred = regionprops_table(label_image=pred_inner_labeled,properties=('centroid',))
centroids_pred = regionprops_table(label_image=pred_both_labeled,properties=('centroid',))
centroids_pred_x = centroids_pred['centroid-0']
centroids_pred_y = centroids_pred['centroid-1']

# stack arrays to 2-dimensional arrays
val_points = np.column_stack((centroids_val_x, centroids_val_y))
pred_points = np.column_stack((centroids_pred_x, centroids_pred_y))

# initialize boolean array which has True values where centroids are for both sets of cells
val_bool_array = np.zeros_like(pred_both_labeled, dtype=bool)
for yx in val_points:
    val_bool_array[round(yx[0]),round(yx[1])]=True

pred_bool_array = np.zeros_like(pred_both_labeled, dtype=bool)
for yx in pred_points:
    pred_bool_array[round(yx[0]),round(yx[1])]=True

# label centroid bool arrays by iterating through them
bool_array_val_labeled,n = ndi.label(val_bool_array)
bool_array_pred_labeled,n_pred = ndi.label(pred_bool_array)

# iterate through validation cells, delete ones that do not have a matching predicted cell
for id in np.unique(bool_array_val_labeled):
    if id == 0:
        continue
    y,x = np.argwhere(bool_array_val_labeled==id)[0]
    # if this statement is true, the cell has not been found. 
    # therefore, we exclude it from further validation and flood fill it with 0's in validation data.
    if pred_both_labeled[y,x] == 0:
        val_both_labeled = skimage.segmentation.flood_fill(val_both_labeled,
                                                          (y,x),
                                                          new_value=0,
                                                          )
        print('deleted cell in validation data (found a false negative): ' + str(id))
        n_false_negative += 1

# iterate through predicted cells, delete ones that do not have a matching predicted cell
for id in np.unique(bool_array_pred_labeled):
    if id == 0:
        continue
    y,x = np.argwhere(bool_array_pred_labeled==id)[0]
    
    if val_both_labeled[y,x]==0:# if there is no cell to test against, delete cell
        pred_both_labeled = skimage.segmentation.flood_fill(image=pred_both_labeled,
                                                       seed_point=(y,x),
                                                       new_value=0,
                                                       )
        print('deleted cell in prediction data (found a false positive): ' + str(id))
        n_false_positive +=1


In [ ]:
# val_both_labeled_temp, n = ndi.label(val_both_labeled)
# pred_both_labeled_temp, n2 = ndi.label(pred_both_labeled)
# # at this point, both arrays should have the same number of cells:
# print('cells in validation data: ' + str(n))
# print('cells in predicted data: ' + str(n2))

In [ ]:
# Now, we need to label each specific cell with the same label in prediction and validation arrays.
# To do that, we iterate through the prediction array and label the validation cells with the same number.
# We do this by flood filling from each centroid.
for id in np.unique(bool_array_val_labeled):
    if id == 0:
        continue
    y,x = np.argwhere(bool_array_val_labeled==id)[0]

    # if there is no cell at the point want to fill it in, delete cell altogether
    if pred_both_labeled[y,x]==0: # if this is true: delete cell
        val_both_labeled = skimage.segmentation.flood_fill(image=val_both_labeled,
                                                           seed_point=(y,x),
                                                           new_value=0,
                                                          )
        print("cell deleted: " + str(id) + str(" - found false negative"))
        n_false_negative += 1
        continue

    # if there is a cell, fill it in with the same label as in mask_val_labeled
    pred_both_labeled = skimage.segmentation.flood_fill(image=pred_both_labeled,
                                                   seed_point=(y,x),
                                                   new_value=id,
                                                   )
    
    # if there is a cell, fill it in with the same label as in mask_val_labeled
    val_both_labeled = skimage.segmentation.flood_fill(image=val_both_labeled,
                                                   seed_point=(y,x),
                                                   new_value=id,
                                                   )



In [ ]:
bool_array_labeled_dil = ndi.binary_dilation(bool_array_val_labeled,iterations=10)

plt.axis("off")
plt.imshow(pred_both_labeled,cmap=glasbey,interpolation='None',vmin=0.1)
plt.imshow(np.ma.array(bool_array_labeled_dil, mask=bool_array_labeled_dil==0), interpolation='none', cmap=glasbey,vmin=0.1)
plt.show()

plt.axis("off")
plt.imshow(val_both_labeled,cmap=glasbey,interpolation='None',vmin=0.1)
plt.imshow(np.ma.array(bool_array_labeled_dil, mask=bool_array_labeled_dil==0), interpolation='none', cmap=glasbey,vmin=0.1)
plt.show()

# plt.imshow(ndi.binary_dilation(bool_array_labeled,iterations=10))
# plt.show()

In [ ]:
print(len(np.unique(val_both_labeled)))
print(len(np.unique(pred_both_labeled)))

In [ ]:
#seperate inner and outer channels back from the combined pictures
pred_inner_labeled = np.copy(pred_both_labeled)
pred_inner_labeled[pred_inner==False]=0

pred_outer_labeled = np.copy(pred_both_labeled)
pred_outer_labeled[pred_outer==False]=0

val_inner_labeled = np.copy(val_both_labeled)
val_inner_labeled[val_inner==False] = 0

val_outer_labeled = np.copy(val_both_labeled)
val_outer_labeled[val_outer==False] = 0

In [ ]:
print(len(np.unique(pred_outer_labeled)))
print(len(np.unique(pred_inner_labeled)))
print(len(np.unique(val_outer_labeled)))
print(len(np.unique(val_inner_labeled)))

In [ ]:
#plot all four channels in 2x2 grid
fig, axs = plt.subplots(ncols=2,nrows=2, figsize=(12,12))
axs[0,0].imshow(pred_outer_labeled[0:1000,0:1000], cmap=glasbey, interpolation='None',vmin=0.1,vmax=pred_outer_labeled.max())
axs[0,1].imshow(pred_inner_labeled[0:1000,0:1000], cmap=glasbey, interpolation='None',vmin=0.1,vmax=pred_outer_labeled.max())
axs[1,0].imshow(val_outer_labeled[0:1000,0:1000], cmap=glasbey, interpolation='None',vmin=0.1,vmax=pred_outer_labeled.max())
axs[1,1].imshow(val_inner_labeled[0:1000,0:1000], cmap=glasbey, interpolation='None',vmin=0.1,vmax=pred_outer_labeled.max())
# Remove axis for each subplot
axs[0, 0].axis('off')
axs[0, 1].axis('off')
axs[1, 0].axis('off')
axs[1, 1].axis('off')
axs[0,0].set_title('Prediction: Outer')
axs[0,1].set_title('Prediction: Inner')
axs[1,0].set_title('Validation: Outer')
axs[1,1].set_title('Validation: Inner')
plt.tight_layout(h_pad=2)
plt.savefig(save_path + "\predict_val_comparison.png",dpi=500)
plt.show()

In [ ]:
# plot overlay of inner + outer
# generate 2px outline of inner and outer area
val_outer_eroded = ndi.binary_erosion(val_outer_labeled,structure=disk(3))
ero = np.logical_xor(val_outer_labeled.astype('bool'),val_outer_eroded)

plt.figure(figsize=(12,4))
plt.title('Validation')
plt.imshow(np.ma.array(val_inner_labeled + val_outer_labeled, mask=(val_inner_labeled + val_outer_labeled) == 0), cmap=glasbey,interpolation='none',vmin=0.1,vmax=pred_outer_labeled.max())
plt.imshow(np.ma.array(ero, mask=(ero) == 0), interpolation='None', cmap='gray',)
plt.axis('off')
plt.savefig(save_path + "val_labeled_cells_outlined.png",dpi=500)
plt.show()

In [ ]:
# plot overlay of inner + outer
# generate 2px outline of inner and outer area
pred_outer_eroded = ndi.binary_erosion(pred_outer_labeled,structure=disk(3))
ero = np.logical_xor(pred_outer_labeled.astype('bool'),pred_outer_eroded)

plt.figure(figsize=(12,4))
plt.title('Prediction')
plt.imshow(np.ma.array(pred_inner_labeled + pred_outer_labeled, mask=(pred_inner_labeled + pred_outer_labeled) == 0), cmap=glasbey,interpolation='none',vmin=0.1,vmax=pred_outer_labeled.max())
plt.imshow(np.ma.array(ero, mask=(ero) == 0), interpolation='None', cmap='gray',)
plt.axis('off')
plt.savefig(save_path + "pred_labeled_cells_outlined.png",dpi=500)
plt.show()

In [ ]:
plt.axis("off")
plt.imshow(pred_both_labeled,cmap=glasbey,interpolation='None',vmin=0.1,vmax=np.max(pred_both_labeled))
# plt.imshow(np.ma.array(bool_array_val_labeled, mask=bool_array_val_labeled==0), interpolation='none', cmap='tab20')
plt.show()

plt.axis("off")
plt.imshow(val_both_labeled,cmap=glasbey,interpolation='None',vmin=0.1,vmax=np.max(pred_both_labeled))
# plt.imshow(np.ma.array(bool_array_val_labeled, mask=bool_array_val_labeled==0), interpolation='none', cmap='tab20')
plt.show()

In [ ]:
# here, we hard filter every cell that does not have a corresponding val or pred cell. Mostly, these are single pixel cells

# iterate through np.unique(pred_outer_labeled), delete all cells that do not have corresponding inner or outer
# also delete every cell that does not have a corresponding val or pred cell
for id in np.unique(pred_outer_labeled):
    # check if id is in val_outer_labeled
    if id not in val_outer_labeled:
        # if not, delete in pred_outer_labeled
        pred_outer_labeled[pred_outer_labeled==id] = 0
        print("deleted outer prediction label #" + str(id))
        n_false_positive += 1

for id in np.unique(val_outer_labeled):
    if id not in pred_outer_labeled:
        # if not, delete in pred_outer_labeled
        val_outer_labeled[val_outer_labeled==id] = 0
        print("deleted outer validation label #" + str(id))
        n_false_negative += 1

for id in np.unique(pred_inner_labeled):
    if id not in val_inner_labeled:
        # if not, delete in pred_outer_labeled
        pred_inner_labeled[pred_inner_labeled==id] = 0
        print("deleted inner prediction label #" + str(id))
        n_false_positive += 1


for id in np.unique(val_inner_labeled):
    if id not in pred_inner_labeled:
        # if not, delete in pred_outer_labeled
        val_inner_labeled[val_inner_labeled==id] = 0
        print("deleted inner validation label #" + str(id))
        n_false_negative += 1


In [ ]:
# iterate through np.unique(pred_inner_labeled), delete all cells that do not have axon or myelin.
for id in np.unique(pred_inner_labeled):
    # check if id is in pred_outer_labeled
    if id not in pred_outer_labeled:
        # if not, delete in pred_inner_labeled
        pred_inner_labeled[pred_inner_labeled==id] = 0
        val_inner_labeled[val_inner_labeled==id] = 0
        print("deleted inner label #" + str(id))
        n_labeled_without_outer += 1

for id in np.unique(pred_outer_labeled):
    if id not in pred_inner_labeled:
        # if not, delete in pred_outer_labeled
        pred_outer_labeled[pred_outer_labeled==id] = 0
        val_outer_labeled[val_outer_labeled==id] = 0
        print("deleted outer label #" + str(id))
        n_labeled_without_inner += 1

In [ ]:
# if there are multiple independent labels with the same labelling number, keep only the biggest one - Inner
for id in np.unique(pred_inner_labeled):
    if id == 0:
        continue
    current_id_mask = pred_inner_labeled==id 
    mask_labeled, n = ndi.label(current_id_mask)
    if n > 1:
        #delete all but the biggest one from this image and from pred_inner_labeled
        current_id_kept = keep_largest_structure(current_id_mask)
        #correct the value of the boolean array to the current id
        current_id_kept = (current_id_kept*id).astype('uint16')
        #re-insert the remaining one to pred_inner_labeled
        pred_inner_labeled[pred_inner_labeled==id] = 0
        pred_inner_labeled = pred_inner_labeled + current_id_kept        
        print ("Removed " + str(n-1) + " duplicates from id #" + str(id))


In [ ]:
# if there are multiple independent labels with the same labelling number, keep only the biggest one - Outer
for id in np.unique(pred_outer_labeled):
    if id == 0:
        continue
    current_id_mask = pred_outer_labeled==id 
    mask_labeled, n = ndi.label(current_id_mask)
    if n > 1:
        #delete all but the biggest one from this image and from pred_inner_labeled
        current_id_kept = keep_largest_structure(current_id_mask)
        #correct the value of the boolean array to the current id
        current_id_kept = (current_id_kept*id).astype('uint16')
        #re-insert the remaining one to pred_inner_labeled
        pred_outer_labeled[pred_outer_labeled==id] = 0
        pred_outer_labeled = pred_outer_labeled + current_id_kept        
        print ("Removed " + str(n-1) + " duplicates from id #" + str(id))


In [ ]:
print(len(np.unique(pred_outer_labeled)))
print(len(np.unique(pred_inner_labeled)))
print(len(np.unique(val_outer_labeled)))
print(len(np.unique(val_inner_labeled)))

In [ ]:
# print(np.unique(pred_inner_labeled) not in np.unique(val_inner_labeled))
# for id in np.unique(pred_inner_labeled):
#     if id in val_outer_labeled:
#         continue
#     print(str(id))
# np.argwhere(pred_inner_labeled==181)

In [ ]:
tf.imwrite(save_path + "/pred_inner_labeled.tif",pred_inner_labeled)
tf.imwrite(save_path + "/pred_outer_labeled.tif",pred_outer_labeled)
tf.imwrite(save_path + "/val_inner_labeled.tif",val_inner_labeled)
tf.imwrite(save_path + "/val_outer_labeled.tif",val_outer_labeled)
tf.imwrite(save_path + "/bool_array_labeled.tif",bool_array_labeled_dil)

In [ ]:
### calculate IoU scores
iou_inner = calculate_iou(pred_inner_labeled.astype('bool'),val_inner_labeled.astype('bool'))
iou_outer = calculate_iou(pred_outer_labeled.astype('bool'),val_outer_labeled.astype('bool'))
### calculate Dice Coefficients / F1 score
dice_inner = calculate_dice(pred_inner_labeled.astype('bool'),val_inner_labeled.astype('bool'))
dice_outer = calculate_dice(pred_outer_labeled.astype('bool'),val_outer_labeled.astype('bool'))

In [ ]:
# plot centroids, measurements, major and minor axis on image to sanity check
regions = regionprops(pred_inner_labeled)

fig, ax = plt.subplots(figsize=(8,8))
ax.imshow(pred_inner_labeled.astype('bool'), cmap=plt.cm.gray)

for props in regions:
    y0, x0 = props.centroid
    orientation = props.orientation
    x1 = x0 + m.cos(orientation) * 0.5 * props.axis_minor_length
    y1 = y0 - m.sin(orientation) * 0.5 * props.axis_minor_length
    x2 = x0 - m.sin(orientation) * 0.5 * props.axis_major_length
    y2 = y0 - m.cos(orientation) * 0.5 * props.axis_major_length

    ax.plot((x0, x1), (y0, y1), '-r', linewidth=5)
    ax.plot((x0, x2), (y0, y2), '-r', linewidth=5)
    ax.plot(x0, y0, '.k', markersize=30)

    minr, minc, maxr, maxc = props.bbox
    bx = (minc, maxc, maxc, minc, minc)
    by = (minr, minr, maxr, maxr, minr)
    ax.plot(bx, by, '-c', linewidth=5)

ax.set_axis_off()
ax.axis((0, 1000, 1000, 0))
plt.savefig(save_path + "\inner_clocks.png",dpi=500)
plt.show()

In [ ]:
## now that we have the same amount of cells and all cells have the same label in both pictures, we extract measurements for validation
table_inner_pred = regionprops_table(label_image=pred_inner_labeled,
                          properties=('label',
                                      'area',
                                      'centroid',
                                      'axis_major_length',
                                      'axis_minor_length',
                                      'eccentricity',
                                      'extent',
                                      'equivalent_diameter_area',
                                      'orientation',
                                      'slice',))
measurements_inner_pred = pd.DataFrame(table_inner_pred)
#measurements_inner_pred.drop(index=66,inplace=True)

table_inner_val = regionprops_table(label_image=val_inner_labeled,
                              properties=('label',
                                          'area',
                                          'centroid',
                                          'axis_major_length',
                                          'axis_minor_length',
                                          'eccentricity',
                                          'extent',
                                          'equivalent_diameter_area',
                                          'orientation',
                                          'slice',))
measurements_inner_val = pd.DataFrame(table_inner_val)
#measurements_inner_val.drop(index=66,inplace=True)

# drop uninteresting columns for now
measurements_inner_val=measurements_inner_val.drop(columns=['label','centroid-0','centroid-1','extent','equivalent_diameter_area','orientation','slice'])
measurements_inner_pred=measurements_inner_pred.drop(columns=['label','centroid-0','centroid-1','extent','equivalent_diameter_area','orientation','slice'])

# calculate mean and std of all columns for both dataframes
stats_inner_pred = pd.concat([measurements_inner_pred.mean(),measurements_inner_pred.median(), measurements_inner_pred.std()], axis=1)
stats_inner_pred.columns = ['mean','median', 'std']
stats_inner_pred['dataset'] = 'prediction'

stats_inner_val = pd.concat([measurements_inner_val.mean(),measurements_inner_val.median(), measurements_inner_val.std()], axis=1)
stats_inner_val.columns = ['mean','median', 'std']
stats_inner_val['dataset'] = 'manual label'

# combine the two datasets
stats = pd.concat([stats_inner_pred, stats_inner_val])

# plot the data as a box plot
fig, axs = plt.subplots(ncols=measurements_inner_pred.shape[1], figsize=(9, 5),)

for i, col in enumerate(measurements_inner_pred.columns):
    ax = axs[i]
    col_stats = stats.loc[stats.index == col, :]
    xticks = np.arange(len(col_stats))
    ax.errorbar(xticks, col_stats['mean'], yerr=col_stats['std'], fmt='o', label='Mean')
    ax.errorbar(xticks, col_stats['median'], fmt='s', label='Median')
    ax.set_xticks(xticks)
    ax.set_xticklabels(col_stats['dataset'])
    ax.set_title(col)
    if i == 0:
        ax.set_ylabel('Value')
        ax.set_ylim(-2000,11000)
    if i == 1:
        ax.set_ylim(30,220)
    if i == 2:
        ax.set_ylim(10,150)
    if i == 3:
        ax.set_ylim(0.5,1)
    
plt.tight_layout()
plt.savefig(save_path + "\inner_lineplot.png",dpi=500)
plt.show()

In [ ]:
print('predicted stats:')
print(stats_inner_pred)
print('')
print('validation stats:')
print(stats_inner_val)

In [ ]:
x = np.arange(0,2)
y = np.arange(0,2)
set_val = (measurements_inner_val['eccentricity'])
set_pred = (measurements_inner_pred['eccentricity'])

stats_val_pred = scipy.stats.linregress(set_val,set_pred)

plt.scatter(set_val,set_pred,marker='.')
plt.plot(x,y)
plt.xlabel('validation eccentricity')
plt.ylabel('predicted eccentricity')
plt.savefig(save_path + "\inner_ecc.png",dpi=500)
plt.show()
print(stats_val_pred)

In [ ]:
# Assuming set_val and set_pred are numpy arrays or lists
set_val = np.array(set_val)
set_pred = np.array(set_pred)

# Calculate the mean and difference
mean = np.mean([set_val, set_pred], axis=0)
diff = set_val - set_pred

# Calculate the limits of agreement (mean difference ± 1.96 * standard deviation of differences)
mean_diff = np.mean(diff)
std_diff = np.std(diff)
upper_limit = mean_diff + 1.96 * std_diff
lower_limit = mean_diff - 1.96 * std_diff

# Plotting the Bland-Altman plot
plt.figure(figsize=(8, 6))
plt.scatter(mean, diff, color='black', s=30, marker='.')
plt.axhline(mean_diff, color='red', linestyle='--', linewidth=1)
plt.axhline(upper_limit, color='blue', linestyle='--', linewidth=1)
plt.axhline(lower_limit, color='blue', linestyle='--', linewidth=1)

# Adding labels and titles
plt.xlabel('Mean of Val and Pred')
plt.ylabel('Difference (Val - Pred)')
plt.title('Bland-Altman Plot of Fiber Eccentricity')

# Adding text for limits of agreement
plt.text(mean.mean(), upper_limit + 0.1, 'Upper Limit', color='blue', ha='center')
plt.text(mean.mean(), lower_limit - 0.1, 'Lower Limit', color='blue', ha='center')

# Save plot
plt.savefig(save_path + "\inner_ecc_ba.png",dpi=500)

# Display the plot
plt.show()


In [ ]:
x = np.arange(0,500)
y = np.arange(0,500)
set_val = (measurements_inner_val['axis_major_length'])
set_pred = (measurements_inner_pred['axis_major_length'])

stats_val_pred = scipy.stats.linregress(set_val,set_pred)

plt.scatter(set_val,set_pred,marker='.')
plt.plot(x,y)
plt.xlabel('validation axis_major_length')
plt.ylabel('predicted axis_major_length')
#plt.ylim(-20,520)
#plt.xlim(-20,520)
plt.savefig(save_path + "\inner_major.png",dpi=500)
plt.show()
print(stats_val_pred)

In [ ]:
set_val = np.array(set_val)
set_pred = np.array(set_pred)

# Calculate the mean and difference
mean = np.mean([set_val, set_pred], axis=0)
diff = set_val - set_pred

# Calculate the limits of agreement (mean difference ± 1.96 * standard deviation of differences)
mean_diff = np.mean(diff)
std_diff = np.std(diff)
upper_limit = mean_diff + 1.96 * std_diff
lower_limit = mean_diff - 1.96 * std_diff

# Plotting the Bland-Altman plot
plt.figure(figsize=(8, 6))
plt.scatter(mean, diff, color='black', s=30,marker='.')
plt.axhline(mean_diff, color='red', linestyle='--', linewidth=1)
plt.axhline(upper_limit, color='blue', linestyle='--', linewidth=1)
plt.axhline(lower_limit, color='blue', linestyle='--', linewidth=1)

# Adding labels and titles
plt.xlabel('Mean of Val and Pred')
plt.ylabel('Difference (Val - Pred)')
plt.title('Bland-Altman Plot of Fiber Major Axis Length')

# Adding text for limits of agreement
plt.text(mean.mean(), upper_limit + 0.1, 'Upper Limit', color='blue', ha='center')
plt.text(mean.mean(), lower_limit - 0.1, 'Lower Limit', color='blue', ha='center')

# Display the plot
plt.show()


In [ ]:
x = np.arange(0,200)
y = np.arange(0,200)
set_val = (measurements_inner_val['axis_minor_length'])
set_pred = (measurements_inner_pred['axis_minor_length'])

stats_val_pred = scipy.stats.linregress(set_val,set_pred)

plt.xticks([0, 50, 100, 150, 200])
plt.yticks([0, 50, 100, 150, 200])

plt.tick_params(axis='both', which='major', labelsize=14)  # You can adjust the font size as needed

plt.scatter(set_val,set_pred,marker='.')
plt.plot(x,y)
plt.xlabel('validation axis_minor_length')
plt.ylabel('predicted axis_minor_length')
plt.savefig(save_path + "\inner_minor.png",dpi=500)
plt.show()
print(stats_val_pred)

In [ ]:
# Assuming set_val and set_pred are numpy arrays or lists
set_val = np.array(set_val)
set_pred = np.array(set_pred)

# Calculate the mean and difference
mean = np.mean([set_val, set_pred], axis=0)
diff = set_val - set_pred

# Calculate the limits of agreement (mean difference ± 1.96 * standard deviation of differences)
mean_diff = np.mean(diff)
std_diff = np.std(diff)
upper_limit = mean_diff + 1.96 * std_diff
lower_limit = mean_diff - 1.96 * std_diff

# Plotting the Bland-Altman plot
plt.figure(figsize=(8, 6))
plt.scatter(mean, diff, color='black', s=30, marker='.')
plt.axhline(mean_diff, color='red', linestyle='--', linewidth=1)
plt.axhline(upper_limit, color='blue', linestyle='--', linewidth=1)
plt.axhline(lower_limit, color='blue', linestyle='--', linewidth=1)

# Adding labels and titles
plt.xlabel('Mean of Val and Pred')
plt.ylabel('Difference (Val - Pred)')
plt.title('Bland-Altman Plot of Fiber Minor Length')

# Adding text for limits of agreement
plt.text(mean.mean(), upper_limit + 0.1, 'Upper Limit', color='blue', ha='center')
plt.text(mean.mean(), lower_limit - 0.1, 'Lower Limit', color='blue', ha='center')

# Save plot
plt.savefig(save_path + "\inner_minor_ba.png",dpi=500)

# Display the plot
plt.show()


In [ ]:
x = np.arange(0,50000)
y = np.arange(0,50000)
set_val = (measurements_inner_val['area'])
set_pred = (measurements_inner_pred['area'])

stats_val_pred = scipy.stats.linregress(set_val,set_pred)

plt.scatter(set_val,set_pred,marker='.')
plt.plot(x,y)
plt.xlabel('validation area')
plt.ylabel('predicted area')
plt.savefig(save_path + "\inner_area.png",dpi=500)
plt.show()
print(stats_val_pred)

In [ ]:
# Assuming set_val and set_pred are numpy arrays or lists
set_val = np.array(set_val)
set_pred = np.array(set_pred)

# Calculate the mean and difference
mean = np.mean([set_val, set_pred], axis=0)
diff = set_val - set_pred

# Calculate the limits of agreement (mean difference ± 1.96 * standard deviation of differences)
mean_diff = np.mean(diff)
std_diff = np.std(diff)
upper_limit = mean_diff + 1.96 * std_diff
lower_limit = mean_diff - 1.96 * std_diff

# Plotting the Bland-Altman plot
plt.figure(figsize=(8, 6))
plt.scatter(mean, diff, color='black', s=30, marker='.')
plt.axhline(mean_diff, color='red', linestyle='--', linewidth=1)
plt.axhline(upper_limit, color='blue', linestyle='--', linewidth=1)
plt.axhline(lower_limit, color='blue', linestyle='--', linewidth=1)

# Adding labels and titles
plt.xlabel('Mean of Val and Pred')
plt.ylabel('Difference (Val - Pred)')
plt.title('Bland-Altman Plot of Fiber Area')

# Adding text for limits of agreement
plt.text(mean.mean(), upper_limit + 0.1, 'Upper Limit', color='blue', ha='center')
plt.text(mean.mean(), lower_limit - 0.1, 'Lower Limit', color='blue', ha='center')

# Save plot
plt.savefig(save_path + "\inner_area_ba.png",dpi=500)

# Display the plot
plt.show()


In [ ]:
# ### this code block is not final. need fo find a good solution for this problem.

# # iterate through np.unique(pred_outer_labeled), delete all cells that do not have axon or myelin.
# for id in np.unique(pred_outer_labeled):
#     # check if id is in pred_outer_labeled
#     if id not in val_outer_labeled:
#         # if not, delete in pred_inner_labeled
#         pred_outer_labeled[pred_outer_labeled==id] = 0
#         print("deleted prediction label #" + str(id))
    

# for id in np.unique(val_outer_labeled):
#     if id not in pred_outer_labeled:
#         # if not, delete in pred_outer_labeled
#         val_outer_labeled[val_outer_labeled==id] = 0
#         print("deleted validation label #" + str(id))


In [ ]:
##### now do the same as above for the outer channel
# plot centroids, measurements, major and minor axis on image to sanity check
regions = regionprops(pred_outer_labeled)

fig, ax = plt.subplots(figsize=(8,8))
ax.imshow(pred_outer_labeled.astype('bool'), cmap=plt.cm.gray)

for props in regions:
    y0, x0 = props.centroid
    orientation = props.orientation
    x1 = x0 + m.cos(orientation) * 0.5 * props.axis_minor_length
    y1 = y0 - m.sin(orientation) * 0.5 * props.axis_minor_length
    x2 = x0 - m.sin(orientation) * 0.5 * props.axis_major_length
    y2 = y0 - m.cos(orientation) * 0.5 * props.axis_major_length

    ax.plot((x0, x1), (y0, y1), '-r', linewidth=5)
    ax.plot((x0, x2), (y0, y2), '-r', linewidth=5)
    ax.plot(x0, y0, '.w', markersize=30)

    minr, minc, maxr, maxc = props.bbox
    bx = (minc, maxc, maxc, minc, minc)
    by = (minr, minr, maxr, maxr, minr)
    ax.plot(bx, by, '-c', linewidth=5)

ax.set_axis_off()
ax.axis((0, 1000, 1000, 0))
plt.savefig(save_path + "\outer_clocks.png",dpi=500)
plt.show()

In [ ]:
## do the same regression analysis as before
# visually compare outer_labeled and validation data
fig, axs = plt.subplots(ncols=2,figsize=(12,12))
axs[0].imshow(pred_outer_labeled, cmap=glasbey,interpolation='none',vmin=0.1,vmax=pred_outer_labeled.max())
axs[1].imshow(val_outer_labeled, cmap=glasbey,interpolation='none',vmin=0.1,vmax=pred_outer_labeled.max())
axs[0].set_title('Outer - prediction')
axs[1].set_title('Outer - validation')
plt.show()

In [ ]:
## now that we have the same amount of cells and all cells have the same label in both pictures, we extract measurements for validation
table_outer_pred = regionprops_table(label_image=pred_outer_labeled,
                          properties=('label',
                                      'area',
                                      'centroid',
                                      'axis_major_length',
                                      'axis_minor_length',
                                      'eccentricity',
                                      'extent',
                                      'equivalent_diameter_area',
                                      'orientation',
                                      'slice'))
measurements_outer_pred = pd.DataFrame(table_outer_pred)
#measurements_outer_pred.drop(66,inplace=True)

table_outer_val = regionprops_table(label_image=val_outer_labeled,
                              properties=('label',
                                          'area',
                                          'centroid',
                                          'axis_major_length',
                                          'axis_minor_length',
                                          'eccentricity',
                                          'extent',
                                          'equivalent_diameter_area',
                                          'orientation',
                                          'slice'))
measurements_outer_val = pd.DataFrame(table_outer_val)
#measurements_outer_val.drop(66,inplace=True)

# drop uninteresting columns for now
measurements_outer_val=measurements_outer_val.drop(columns=['label','centroid-0','centroid-1','extent','equivalent_diameter_area','orientation','slice'])
measurements_outer_pred=measurements_outer_pred.drop(columns=['label','centroid-0','centroid-1','extent','equivalent_diameter_area','orientation','slice'])

# calculate mean and std of all columns for both dataframes
stats_outer_pred = pd.concat([measurements_outer_pred.mean(),measurements_outer_pred.median(), measurements_outer_pred.std()], axis=1)
stats_outer_pred.columns = ['mean','median', 'std']
stats_outer_pred['dataset'] = 'prediction'

stats_outer_val = pd.concat([measurements_outer_val.mean(),measurements_outer_val.median(), measurements_outer_val.std()], axis=1)
stats_outer_val.columns = ['mean','median', 'std']
stats_outer_val['dataset'] = 'manual label'

# combine the two datasets
stats = pd.concat([stats_outer_pred, stats_outer_val])

# plot the data as a box plot
fig, axs = plt.subplots(ncols=measurements_outer_pred.shape[1], figsize=(9, 5),)

for i, col in enumerate(measurements_outer_pred.columns):
    ax = axs[i]
    col_stats = stats.loc[stats.index == col, :]
    xticks = np.arange(len(col_stats))
    ax.errorbar(xticks, col_stats['mean'], yerr=col_stats['std'], fmt='o', label='Mean')
    ax.errorbar(xticks, col_stats['median'], fmt='s', label='Median')
    ax.set_xticks(xticks)
    ax.set_xticklabels(col_stats['dataset'])
    ax.set_title(col)
    if i == 0:
        ax.set_ylabel('Value')
        ax.set_ylim(-2000,11000)
    if i == 1:
        ax.set_ylim(30,220)
    if i == 2:
        ax.set_ylim(10,150)
    if i == 3:
        ax.set_ylim(0.5,1)

# ax.legend(bbox_to_anchor=(2.5,0.5))
plt.tight_layout()
plt.savefig(save_path + "\outer_lineplot.png",dpi=500)
plt.show()

In [ ]:
print('predicted stats:')
print(stats_outer_pred)
print('')
print('validation stats:')
print(stats_outer_val)

In [ ]:
x = np.arange(0,2)
y = np.arange(0,2)
set_val = (measurements_outer_val['eccentricity'])
set_pred = (measurements_outer_pred['eccentricity'])

stats_val_pred = scipy.stats.linregress(set_val,set_pred)

plt.scatter(set_val,set_pred,marker='.')
plt.plot(x,y)
plt.xlabel('validation eccentricity')
plt.ylabel('predicted eccentricity')
plt.savefig(save_path + "\outer_ecc.png",dpi=500)
plt.show()
print(stats_val_pred)

In [ ]:
# Assuming set_val and set_pred are numpy arrays or lists
set_val = np.array(set_val)
set_pred = np.array(set_pred)

# Calculate the mean and difference
mean = np.mean([set_val, set_pred], axis=0)
diff = set_val - set_pred

# Calculate the limits of agreement (mean difference ± 1.96 * standard deviation of differences)
mean_diff = np.mean(diff)
std_diff = np.std(diff)
upper_limit = mean_diff + 1.96 * std_diff
lower_limit = mean_diff - 1.96 * std_diff

# Plotting the Bland-Altman plot
plt.figure(figsize=(8, 6))
plt.scatter(mean, diff, color='black', s=30, marker='.')
plt.axhline(mean_diff, color='red', linestyle='--', linewidth=1)
plt.axhline(upper_limit, color='blue', linestyle='--', linewidth=1)
plt.axhline(lower_limit, color='blue', linestyle='--', linewidth=1)

# Adding labels and titles
plt.xlabel('Mean of Val and Pred')
plt.ylabel('Difference (Val - Pred)')
plt.title('Bland-Altman Plot of Myelination Eccentricity')

# Adding text for limits of agreement
plt.text(mean.mean(), upper_limit + 0.1, 'Upper Limit', color='blue', ha='center')
plt.text(mean.mean(), lower_limit - 0.1, 'Lower Limit', color='blue', ha='center')

# Save plot
plt.savefig(save_path + "\outer_ecc_ba.png",dpi=500)

# Display the plot
plt.show()


In [ ]:
x = np.arange(0,500)
y = np.arange(0,500)
set_val = (measurements_outer_val['axis_major_length'])
set_pred = (measurements_outer_pred['axis_major_length'])

stats_val_pred = scipy.stats.linregress(set_val,set_pred)

plt.scatter(set_val,set_pred,marker='.')
plt.plot(x,y)
plt.xlabel('validation axis_major_length')
plt.ylabel('predicted axis_major_length')
plt.savefig(save_path + "\outer_major.png",dpi=500)
plt.show()
print(stats_val_pred)

In [ ]:
# Assuming set_val and set_pred are numpy arrays or lists
set_val = np.array(set_val)
set_pred = np.array(set_pred)

# Calculate the mean and difference
mean = np.mean([set_val, set_pred], axis=0)
diff = set_val - set_pred

# Calculate the limits of agreement (mean difference ± 1.96 * standard deviation of differences)
mean_diff = np.mean(diff)
std_diff = np.std(diff)
upper_limit = mean_diff + 1.96 * std_diff
lower_limit = mean_diff - 1.96 * std_diff

# Plotting the Bland-Altman plot
plt.figure(figsize=(8, 6))
plt.scatter(mean, diff, color='black', s=30, marker='.')
plt.axhline(mean_diff, color='red', linestyle='--', linewidth=1)
plt.axhline(upper_limit, color='blue', linestyle='--', linewidth=1)
plt.axhline(lower_limit, color='blue', linestyle='--', linewidth=1)

# Adding labels and titles
plt.xlabel('Mean of Val and Pred')
plt.ylabel('Difference (Val - Pred)')
plt.title('Bland-Altman Plot of Myelination Major Axis Length')

# Adding text for limits of agreement
plt.text(mean.mean(), upper_limit + 0.1, 'Upper Limit', color='blue', ha='center')
plt.text(mean.mean(), lower_limit - 0.1, 'Lower Limit', color='blue', ha='center')

# Save plot
plt.savefig(save_path + "\outer_major_ba.png",dpi=500)

# Display the plot
plt.show()


In [ ]:
x = np.arange(0,300)
y = np.arange(0,300)
set_val = (measurements_outer_val['axis_minor_length'])
set_pred = (measurements_outer_pred['axis_minor_length'])

stats_val_pred = scipy.stats.linregress(set_val,set_pred)

plt.xticks([0, 50, 100, 150, 200, 250, 300])
plt.yticks([0, 50, 100, 150, 200, 250, 300])

plt.tick_params(axis='both', which='major', labelsize=14)  # You can adjust the font size as needed
plt.scatter(set_val,set_pred,marker='.')
plt.plot(x,y)
plt.xlabel('validation axis_minor_length')
plt.ylabel('predicted axis_minor_length')
plt.savefig(save_path + "\outer_minor.png",dpi=500)
plt.show()
print(stats_val_pred)

In [ ]:
# Assuming set_val and set_pred are numpy arrays or lists
set_val = np.array(set_val)
set_pred = np.array(set_pred)

# Calculate the mean and difference
mean = np.mean([set_val, set_pred], axis=0)
diff = set_val - set_pred

# Calculate the limits of agreement (mean difference ± 1.96 * standard deviation of differences)
mean_diff = np.mean(diff)
std_diff = np.std(diff)
upper_limit = mean_diff + 1.96 * std_diff
lower_limit = mean_diff - 1.96 * std_diff

# Plotting the Bland-Altman plot
plt.figure(figsize=(8, 6))
plt.scatter(mean, diff, color='black', s=30, marker='.')
plt.axhline(mean_diff, color='red', linestyle='--', linewidth=1)
plt.axhline(upper_limit, color='blue', linestyle='--', linewidth=1)
plt.axhline(lower_limit, color='blue', linestyle='--', linewidth=1)

# Adding labels and titles
plt.xlabel('Mean of Val and Pred')
plt.ylabel('Difference (Val - Pred)')
plt.title('Bland-Altman Plot of Myelination Minor Axis Length')

# Adding text for limits of agreement
plt.text(mean.mean(), upper_limit + 0.1, 'Upper Limit', color='blue', ha='center')
plt.text(mean.mean(), lower_limit - 0.1, 'Lower Limit', color='blue', ha='center')

# Save plot
plt.savefig(save_path + "\outer_minor_ba.png",dpi=500)

# Display the plot
plt.show()


In [ ]:
x = np.arange(0,40000)
y = np.arange(0,40000)
set_val = (measurements_outer_val['area'])
set_pred = (measurements_outer_pred['area'])

stats_val_pred = scipy.stats.linregress(set_val,set_pred)

plt.scatter(set_val,set_pred,marker='.')
plt.plot(x,y)
plt.xlabel('validation area')
plt.ylabel('predicted area')
plt.savefig(save_path + "\outer_area.png",dpi=500)
plt.show()
print(stats_val_pred)

In [ ]:
# Assuming set_val and set_pred are numpy arrays or lists
set_val = np.array(set_val)
set_pred = np.array(set_pred)

# Calculate the mean and difference
mean = np.mean([set_val, set_pred], axis=0)
diff = set_val - set_pred

# Calculate the limits of agreement (mean difference ± 1.96 * standard deviation of differences)
mean_diff = np.mean(diff)
std_diff = np.std(diff)
upper_limit = mean_diff + 1.96 * std_diff
lower_limit = mean_diff - 1.96 * std_diff

# Plotting the Bland-Altman plot
plt.figure(figsize=(8, 6))
plt.scatter(mean, diff, color='black', s=30, marker='.')
plt.axhline(mean_diff, color='red', linestyle='--', linewidth=1)
plt.axhline(upper_limit, color='blue', linestyle='--', linewidth=1)
plt.axhline(lower_limit, color='blue', linestyle='--', linewidth=1)

# Adding labels and titles
plt.xlabel('Mean of Val and Pred')
plt.ylabel('Difference (Val - Pred)')
plt.title('Bland-Altman Plot of Myelination Area')

# Adding text for limits of agreement
plt.text(mean.mean(), upper_limit + 0.1, 'Upper Limit', color='blue', ha='center')
plt.text(mean.mean(), lower_limit - 0.1, 'Lower Limit', color='blue', ha='center')

# Save plot
plt.savefig(save_path + "\outer_area_ba.png",dpi=500)

# Display the plot
plt.show()


In [ ]:
# calculate gratio for validation and prediction set, plot them against each other and run linreg
# gratio = inner_minor/outer_minor
x = np.arange(0,2)
y = np.arange(0,2)

gratio_pred = measurements_inner_pred['axis_minor_length']/measurements_outer_pred['axis_minor_length']
gratio_val  = measurements_inner_val['axis_minor_length']/measurements_outer_val['axis_minor_length']

stats_val_pred = scipy.stats.linregress(gratio_val,gratio_pred)

plt.scatter(gratio_val,gratio_pred,marker='.')
plt.plot(x,y)
plt.xlabel('validation gratio')
plt.ylabel('predicted gratio')
plt.savefig(save_path + "\gratio.png",dpi=500)
plt.show()
print(stats_val_pred)

In [ ]:
# Assuming set_val and set_pred are numpy arrays or lists
set_val = np.array(set_val)
set_pred = np.array(set_pred)

# Calculate the mean and difference
mean = np.mean([set_val, set_pred], axis=0)
diff = set_val - set_pred

# Calculate the limits of agreement (mean difference ± 1.96 * standard deviation of differences)
mean_diff = np.mean(diff)
std_diff = np.std(diff)
upper_limit = mean_diff + 1.96 * std_diff
lower_limit = mean_diff - 1.96 * std_diff

# Plotting the Bland-Altman plot
plt.figure(figsize=(8, 6))
plt.scatter(mean, diff, color='black', s=30, marker='.')
plt.axhline(mean_diff, color='red', linestyle='--', linewidth=1)
plt.axhline(upper_limit, color='blue', linestyle='--', linewidth=1)
plt.axhline(lower_limit, color='blue', linestyle='--', linewidth=1)

# Adding labels and titles
plt.xlabel('Mean of Val and Pred')
plt.ylabel('Difference (Val - Pred)')
plt.title('Bland-Altman Plot of g Ratio')

# Adding text for limits of agreement
plt.text(mean.mean(), upper_limit + 0.1, 'Upper Limit', color='blue', ha='center')
plt.text(mean.mean(), lower_limit - 0.1, 'Lower Limit', color='blue', ha='center')

# Save plot
plt.savefig(save_path + "\gratio_ba.png",dpi=500)

# Display the plot
plt.show()


In [ ]:
# print other statistics
print ("Correctly identified cells (prediction): " + str (len(set_pred)))
print ("False Positives (prediction): " + str (n_false_positive))
print ("False Negatives (prediction): " + str (n_false_negative))
print ("Percentage of cells found: " + str(len(set_pred)/(len(set_pred)+n_false_negative))) # found cells divided by found cells + false negatives
print ("Percentage of cells falsely identified: " + str(n_false_positive/(len(set_pred)+n_false_positive)))
print ("Cells removed because of touching border (prediction): " + str(n_cells_removed_border_pred))
print ("Cells removed because of touching border (validation): " + str(n_cells_removed_border_val))
print ("Cells removed because labeled without inner (prediction): " + str (n_labeled_without_inner))
print ("Cells removed because labeled without outer (prediction): " + str (n_labeled_without_outer))
print ("IoU for fibers: " + str (iou_inner))
print ("IoU for myelin: " + str (iou_outer))
print ("dice coefficient / F1 score for fibers: " + str (dice_inner))
print ("dice coefficient / F1 score for myelin: " + str (dice_outer))

In [ ]:
# write everything from above into a file as well
text_to_write = """Correctly identified cells (prediction): {}
False Positives (prediction): {}
False Negatives (prediction): {}
Percentage of cells found: {}
Percentage of cells falsely identified: {}
Cells removed because of touching border (prediction): {}
Cells removed because of touching border (validation): {}
Cells removed because labeled without inner (prediction): {}
Cells removed because labeled without outer (prediction): {}
IoU for fibers: {}
IoU for myelin: {}
dice coefficient / F1 score for fibers: {}
dice coefficient / F1 score for myelin: {}""".format(
    len(set_pred),
    n_false_positive,
    n_false_negative,
    len(set_pred)/(len(set_pred)+n_false_negative),
    n_false_positive/(len(set_pred)+n_false_positive),
    n_cells_removed_border_pred,
    n_cells_removed_border_val,
    n_labeled_without_inner,
    n_labeled_without_outer,
    iou_inner,
    iou_outer,
    dice_inner,
    dice_outer
)

# Write the text to the file
with open(os.getcwd() + r"\5_val_results\results.txt", "w") as file:
    file.write(text_to_write)